In [1]:
import numpy as np
import pandas as pd
from hyperopt import STATUS_OK,Trials,fmin,hp,tpe
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import mlflow
from mlflow.models import infer_signature

/home/test-14900/anaconda3/envs/conda-env-3-11/lib/python3.11/site-packages/hyperopt/atpe.py:19: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [2]:
from tensorflow import keras

2026-01-24 05:43:28.582162: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-01-24 05:43:28.692472: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2026-01-24 05:43:29.552119: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [3]:
df=pd.read_csv("https://raw.githubusercontent.com/mlflow/mlflow/master/tests/datasets/winequality-white.csv",sep=";")

In [4]:
df.head(5)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4898 entries, 0 to 4897
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         4898 non-null   float64
 1   volatile acidity      4898 non-null   float64
 2   citric acid           4898 non-null   float64
 3   residual sugar        4898 non-null   float64
 4   chlorides             4898 non-null   float64
 5   free sulfur dioxide   4898 non-null   float64
 6   total sulfur dioxide  4898 non-null   float64
 7   density               4898 non-null   float64
 8   pH                    4898 non-null   float64
 9   sulphates             4898 non-null   float64
 10  alcohol               4898 non-null   float64
 11  quality               4898 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 459.3 KB


In [6]:
df.isna().value_counts()

fixed acidity  volatile acidity  citric acid  residual sugar  chlorides  free sulfur dioxide  total sulfur dioxide  density  pH     sulphates  alcohol  quality
False          False             False        False           False      False                False                 False    False  False      False    False      4898
Name: count, dtype: int64

In [7]:
df.isnull().value_counts()

fixed acidity  volatile acidity  citric acid  residual sugar  chlorides  free sulfur dioxide  total sulfur dioxide  density  pH     sulphates  alcohol  quality
False          False             False        False           False      False                False                 False    False  False      False    False      4898
Name: count, dtype: int64

In [8]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
fixed acidity,4898.0,6.854788,0.843868,3.80000,6.300000,6.80000,7.3000,14.20000
volatile acidity,4898.0,0.278241,0.100795,0.08000,0.210000,0.26000,0.3200,1.10000
citric acid,4898.0,0.334192,0.121020,0.00000,0.270000,0.32000,0.3900,1.66000
residual sugar,4898.0,6.391415,5.072058,0.60000,1.700000,5.20000,9.9000,65.80000
chlorides,4898.0,0.045772,0.021848,0.00900,0.036000,0.04300,0.0500,0.34600
free sulfur dioxide,4898.0,35.308085,17.007137,2.00000,23.000000,34.00000,46.0000,289.00000
total sulfur dioxide,4898.0,138.360657,42.498065,9.00000,108.000000,134.00000,167.0000,440.00000
density,4898.0,0.994027,0.002991,0.98711,0.991723,0.99374,0.9961,1.03898
pH,4898.0,3.188267,0.151001,2.72000,3.090000,3.18000,3.2800,3.82000
sulphates,4898.0,0.489847,0.114126,0.22000,0.410000,0.47000,0.5500,1.08000


In [9]:
quality=df['quality']
display(quality)

0       6
1       6
2       6
3       6
4       6
       ..
4893    6
4894    5
4895    6
4896    7
4897    6
Name: quality, Length: 4898, dtype: int64

In [10]:
df.drop(columns='quality',axis=1)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.00100,3.00,0.45,8.8
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.99400,3.30,0.49,9.5
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.99510,3.26,0.44,10.1
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9
...,...,...,...,...,...,...,...,...,...,...,...
4893,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2
4894,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6
4895,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4
4896,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8


In [11]:

X_train,X_test,y_train,y_test=train_test_split(df,quality,test_size=0.25,random_state=45)

In [12]:
signature=infer_signature(X_train.to_numpy(),y_train)

/home/test-14900/anaconda3/envs/conda-env-3-11/lib/python3.11/site-packages/mlflow/types/utils.py:407: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(


In [13]:
### ANN
def train_model(parameters,epochs,train_x,train_y,test_x,test_y):
    ## model architecture
    mean=np.mean(test_x,axis=0)
    var=np.var(test_x,axis=0)
    model=keras.Sequential(
        [
            keras.Input([train_x.shape[1]]),
            keras.layers.Normalization(mean=mean,variance=var),
            keras.layers.Dense(64,activation="relu"),
            keras.layers.Dense(1)
								]
				)
    model.compile(optimizer=keras.optimizers.SGD(
        learning_rate=parameters["lr"],momentum=parameters["momentum"]
				),
                loss="mean_squared_error",
    metrics=[keras.metrics.RootMeanSquaredError()]
    
				)
    with mlflow.start_run(nested=True):
            model.fit(train_x,train_y,validation_data=(test_x,test_y),epochs=epochs,batch_size=64)
            eval_result=model.evaluate(test_x,test_y,batch_size=64)
            eval_rmse=eval_result[1]
            mlflow.log_params(params=parameters)
            mlflow.log_metric("eval_rmse",eval_rmse)
            mlflow.tensorflow.log_model(model,"model",signature=signature)
            return {"loss":eval_rmse,"status":STATUS_OK,"model":model}

In [14]:
mlflow.set_tracking_uri("http://127.0.0.1:5000")


In [15]:
def objective(params):
    result=train_model(params,50,X_train,y_train,X_test,y_test)
    return result

In [16]:
space={
    "lr":hp.loguniform("lr",np.log(1e-5),np.log(1e-1)),
    "momentum":hp.uniform("momentum",0.0,1.0)
}

In [17]:
mlflow.set_experiment("ANN with mlflow")
with mlflow.start_run():
    # Conduct hyperparam search using Hyperopt
    trials=Trials()
    best=fmin(fn=objective,space=space,algo=tpe.suggest,max_evals=5,trials=trials)
    best_run=sorted(trials.results,key=lambda x:x["loss"])[0]
    mlflow.log_params(best)
    mlflow.log_metric("eval_rmse",best_run['loss'])
    mlflow.tensorflow.log_model(best_run["model"],"model",signature=signature)
    

  0%|          | 0/5 [00:00<?, ?trial/s, best loss=?]

2026-01-24 05:43:32.746615: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


Epoch 1/50                                           

 1/58 ━━━━━━━━━━━━━━━━━━━━ 11s 205ms/step - loss: 35.4452 - root_mean_squared_error: 5.9536
58/58 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2.5217 - root_mean_squared_error: 1.5880 - val_loss: 0.3826 - val_root_mean_squared_error: 0.6186

Epoch 2/50                                           

 1/58 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.4852 - root_mean_squared_error: 0.6966
58/58 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.1641 - root_mean_squared_error: 0.4051 - val_loss: 0.1131 - val_root_mean_squared_error: 0.3363

Epoch 3/50                                           

 1/58 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.0674 - root_mean_squared_error: 0.2597
58/58 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0520 - root_mean_squared_error: 0.2280 - val_loss: 0.0487 - val_root_mean_squared_error: 0.2206

Epoch 4/50                                           

 1/58 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.0312 - root_mean_squar

2026/01/24 05:43:40 INFO mlflow.tracking._tracking_service.client: 🏃 View run intelligent-roo-855 at: http://127.0.0.1:5000/#/experiments/306340854970463397/runs/451b40586bf5437cafb403f8f16f4414.

2026/01/24 05:43:40 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/306340854970463397.



Epoch 1/50                                                                      

 1/58 ━━━━━━━━━━━━━━━━━━━━ 7s 135ms/step - loss: 33.8851 - root_mean_squared_error: 5.8211
58/58 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 30.3359 - root_mean_squared_error: 5.5078 - val_loss: 27.0281 - val_root_mean_squared_error: 5.1989

Epoch 2/50                                                                      

 1/58 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 27.9151 - root_mean_squared_error: 5.2835
58/58 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 24.4961 - root_mean_squared_error: 4.9494 - val_loss: 21.8167 - val_root_mean_squared_error: 4.6708

Epoch 3/50                                                                      

 1/58 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 21.7981 - root_mean_squared_error: 4.6688
58/58 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 19.7180 - root_mean_squared_error: 4.4405 - val_loss: 17.5343 - val_root_mean_squared_error: 4.1874

Epoch 4/50                             

...

2026/01/24 05:44:03 INFO mlflow.tracking._tracking_service.client: 🏃 View run amazing-midge-670 at: http://127.0.0.1:5000/#/experiments/306340854970463397/runs/c6ea1a249c434fffa5921238ab7af9de.

2026/01/24 05:44:03 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/306340854970463397.



Epoch 1/50                                                                      

 1/58 ━━━━━━━━━━━━━━━━━━━━ 7s 140ms/step - loss: 31.9242 - root_mean_squared_error: 5.6502
58/58 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 31.3005 - root_mean_squared_error: 5.5947 - val_loss: 29.1928 - val_root_mean_squared_error: 5.4030

Epoch 2/50                                                                      

 1/58 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 28.8608 - root_mean_squared_error: 5.3722
58/58 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 27.8839 - root_mean_squared_error: 5.2805 - val_loss: 25.9693 - val_root_mean_squared_error: 5.0960

Epoch 3/50                                                                      

 1/58 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 25.3276 - root_mean_squared_error: 5.0327
58/58 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 24.7566 - root_mean_squared_error: 4.9756 - val_loss: 23.0771 - val_root_mean_squared_error: 4.8039

Epoch 4/50                             

2026/01/24 05:44:10 INFO mlflow.tracking._tracking_service.client: 🏃 View run unruly-shrew-743 at: http://127.0.0.1:5000/#/experiments/306340854970463397/runs/7bb04e05d0ac4a40a38a5fdc02ff78fa.

2026/01/24 05:44:10 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/306340854970463397.



100%|██████████| 5/5 [00:39<00:00,  7.84s/trial, best loss: 0.04922802001237869]


2026/01/24 05:44:13 INFO mlflow.tracking._tracking_service.client: 🏃 View run upset-loon-796 at: http://127.0.0.1:5000/#/experiments/306340854970463397/runs/13baceb1309e43369a5cf3f950e53c87.
2026/01/24 05:44:13 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/306340854970463397.


In [18]:
## validating model
import mlflow.pyfunc

model_uri = 'runs:/6d4ec9c4d2244eeb8dba78b599b86e45/model'
loaded_model = mlflow.pyfunc.load_model(model_uri)

RestException: RESOURCE_DOES_NOT_EXIST: Run '6d4ec9c4d2244eeb8dba78b599b86e45' not found

In [ ]:
from mlflow.models import convert_input_example_to_serving_input

In [ ]:
serving_payload = convert_input_example_to_serving_input(X_test)

In [ ]:
from mlflow.models import validate_serving_input

In [ ]:
validate_serving_input(model_uri,serving_payload)

39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 


array([[5.003921 ],
       [5.9681168],
       [6.0304794],
       ...,
       [4.9877253],
       [5.997781 ],
       [6.0241003]], dtype=float32)